In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [2]:
def scaled_dot_product(q, k, v, mask = None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scaled += mask
    attention = F.softmax(scaled, dim = -1)
    values = torch.matmul(attention, v)

    return values, attention

In [3]:
class MultiheadAttention(nn.Module):
    def __init__(self,d_model, num_heads):
        super().__init__()
        self.d_model = d_model  ## attention head embedding size
        assert d_model % num_heads == 0, "d_model should be divisible by num_heads"
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_linear = nn.Linear(d_model, 3 * d_model)
        self.linear_layer = nn.Linear(d_model, d_model)
    
    def forward(self, x, mask = None):
        batch_size, sequence_length, d_model = x.size()
        print(f"x.size(),{x.size()}")
        qkv = self.qkv_linear(x) # (batch_size, seq_len, 3 * d_model)
        print(f"qkv.size(),{qkv.size()}")
        qkv = qkv.reshape(batch_size, sequence_length, self.num_heads, 3*self.head_dim)
        print(f"qkv.size(), {qkv.size()}")
        qkv = qkv.permute(0, 2, 1, 3) # (batch_size, num_heads, seq_len, head_dim)
        print(f"qkv.size(), {qkv.size()}")
        q, k, v = qkv.chunk(3, dim=-1)
        print(f"q.size(), {q.size()}", f"k.size(), {k.size()}", f"v.size(), {v.size()}")
        values, attention_weights = scaled_dot_product(q, k, v, mask)
        print(f"values.size(), {values.size()}", f"attention_weights.size(), {attention_weights.size()}")
        values = values.reshape(batch_size, sequence_length, self.d_model)
        print(f"values.size(), {values.size()}")
        out = self.linear_layer(values)
        print(f"out.size(), {out.size()}")
        return out

In [4]:
class LayerNormalization():
    def __init__(self, parameters_shape, eps=1e-5):
        self.parameters_shape=parameters_shape # [512]
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape)) #learnable parameters [512] sd
        self.beta =  nn.Parameter(torch.zeros(parameters_shape)) #learnable parameters [512] mean

    def forward(self, inputs): # 30 x 200 x 512
        dims = [-(i + 1) for i in range(len(self.parameters_shape))] # [-1] last dimension
        mean = inputs.mean(dim=dims, keepdim=True) # mean of all 512 values per word 30 x 200 x 1
        print(f"Mean \n ({mean.size()}): \n {mean}") 
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)# variance of all 512 values per word 30 x 200 x 1
        std = (var + self.eps).sqrt() #sd
        print(f"Standard Deviation \n ({std.size()}): \n {std}")
        y = (inputs - mean) / std # 30 x 200 x 512
        print(f"y \n ({y.size()}) = \n {y}")
        out = self.gamma * y  + self.beta
        print(f"out \n ({out.size()}) = \n {out}")
        return out

In [5]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden) #512 x 2048
        self.linear2 = nn.Linear(hidden, d_model) #2048 x 512
        self.relu = nn.ReLU()       
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):   # 30 x 200 x 512
        x = self.linear1(x) # 30 x 200 x 2048
        x = self.relu(x)    # 30 x 200 x 2048
        x = self.dropout(x) # 30 x 200 x 2048
        x = self.linear2(x) # 30 x 200 x 512
        return x

In [6]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiheadAttention(d_model=d_model, num_heads = num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden = ffn_hidden, drop_prob = drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x):
        residual_x = x
        x = self.self_attn(x, mask=None) # 30 x 200 x 512
        x = self.dropout1(x) # 30 x 200 x 512
        x = self.norm1(x + residual_x) # 30 x 200 x 512
        residual_x = x
        x = self.ffn(x) 
        x = self.dropout2(x)
        x = self.norm2(x + residual_x)
        return x

In [7]:
class Encoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers):
        super().__init__()
        self.layers = nn.ModuleList(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob) 
                                      for _ in range(num_layers)]) # num_layers개의 EncoderLayer를 ���음

    def forward(self, x):
        return self.layers(x)